In [283]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [284]:
df=pd.read_csv("./data/train.csv")
# For use while saving Out.csv file

In [285]:
df=df.drop(columns=["PassengerId","Ticket","Name","Cabin"])
df=df.fillna(0)


In [286]:
df["Embarked"].unique()

array(['S', 'C', 'Q', 0], dtype=object)

In [287]:
catgry_params=["Sex","Embarked"]
df=pd.get_dummies(data=df,columns=catgry_params,dtype=int)

In [288]:
y_train=torch.tensor(df["Survived"], dtype=torch.float64, device="cuda").unsqueeze(dim=1)
x_train=df.drop(columns=["Survived"])

x_train=torch.tensor(x_train.values, dtype=torch.float64, device="cuda")

In [289]:
# Build the class
model=nn.Sequential(
    nn.Linear(in_features=x_train.shape[1],out_features=x_train.shape[1]*2, device="cuda", dtype=torch.float64),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(in_features=2*x_train.shape[1],out_features=1,device="cuda", dtype=torch.float64),
    nn.Sigmoid(),
    nn.Dropout(p=0.01)
)

In [290]:
# Build training loop
epochs=500000
loss=nn.BCELoss()
optimizer=torch.optim.SGD(model.parameters(), lr=0.001)

In [291]:
for epoch in range(1,epochs+1):
    model.train()
    optimizer.zero_grad()
    y_pred=model(x_train).to("cuda")
    y_pred = torch.clamp(y_pred, min=1e-7, max=1-1e-7)
    # print(y_pred.unique())
    _loss=loss(y_pred, y_train)
    _loss.backward()
    optimizer.step()

In [292]:
model.eval()
with torch.inference_mode():
    y_pred=model(x_train)
    # implement threshold
    y_pred=[1 if y>=0.5 else 0 for y in y_pred]
    print(sum([1 if y_pred[i]==y_train[i] else 0 for i in range(len(y_pred))])/len(y_train))

model.state_dict()

0.8338945005611672


OrderedDict([('0.weight',
              tensor([[ 1.8348e-01, -1.1284e-01,  6.4873e-02,  1.6201e-01,  1.8548e-01,
                        3.4575e-01, -1.3668e-02,  1.7379e-01,  1.4363e-01, -2.6426e-01,
                        1.7388e-01],
                      [-6.7856e-02,  6.2073e-02,  9.3696e-03, -5.7098e-02, -4.8425e-02,
                       -2.5731e-01,  3.4951e-01, -2.0878e-01, -1.0436e-02,  9.4485e-02,
                       -3.4198e-02],
                      [ 4.0433e-02, -1.7228e-01, -2.6415e-01, -1.4761e-02, -5.4457e-01,
                        1.6312e-01,  1.6875e-01, -2.5495e-01, -1.7494e-01, -1.0229e-02,
                        4.0383e-01],
                      [ 3.3499e-01,  1.5971e-02,  2.0298e-01,  1.1161e-01, -6.8805e-03,
                       -6.2108e-01,  6.8786e-02,  2.1835e-01, -4.1387e-01, -1.2373e-02,
                       -2.6729e-01],
                      [-5.6448e-01,  6.9320e-02, -1.1296e-01,  2.6159e-01, -3.0285e-01,
                        1.2508e-01

In [293]:
df2=pd.read_csv('data/test.csv')
PassengerId=df2["PassengerId"]
df2=df2.drop(columns=["PassengerId","Ticket","Name","Cabin"])
print(df2["Embarked"].unique())
df2=df2.fillna(0)
df2=pd.get_dummies(data=df2,columns=catgry_params,dtype=int)

['Q' 'S' 'C']


In [294]:
x_train.shape

torch.Size([891, 11])

In [295]:
df.drop(columns=["Survived"])

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_0,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,0,0,1
1,1,38.0,1,0,71.2833,1,0,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0,1,0,0,0,1
887,1,19.0,0,0,30.0000,1,0,0,0,0,1
888,3,0.0,1,2,23.4500,1,0,0,0,0,1
889,1,26.0,0,0,30.0000,0,1,0,1,0,0


In [296]:
order=list(df.drop(columns=["Survived"]).columns)
idx=order.index("Embarked_0")
df2.insert(idx,"Embarked_0",[0]*len(df2))

In [297]:
_list=df2.to_numpy()
x_test=torch.tensor(data=_list,device="cuda",dtype=torch.float64)

In [298]:
model.eval()
with torch.inference_mode():
    y_pred=model(x_test)
    y_pred=[1 if y>=0.5 else 0 for y in y_pred]
    df2["Survived"]=y_pred
    df2["PassengerId"]=PassengerId

In [299]:
df2.to_csv(path_or_buf="data/Out.csv", columns=["Survived", "PassengerId"], index=False)

In [300]:
print(pd.read_csv('data/Out.csv'))
PassengerId

     Survived  PassengerId
0           0          892
1           0          893
2           0          894
3           0          895
4           1          896
..        ...          ...
413         0         1305
414         1         1306
415         0         1307
416         0         1308
417         0         1309

[418 rows x 2 columns]


0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [301]:
df2.describe()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_0,Embarked_C,Embarked_Q,Embarked_S,Survived,PassengerId
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.0,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,24.044258,0.447368,0.392344,35.541956,0.363636,0.636364,0.0,0.244019,0.110048,0.645933,0.366029,1100.500000
std,0.841838,17.599608,0.896760,0.981429,55.867684,0.481622,0.481622,0.0,0.430019,0.313324,0.478803,0.482295,120.810458
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,892.000000
25%,1.000000,9.000000,0.000000,0.000000,7.895800,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,996.250000
50%,3.000000,24.000000,0.000000,0.000000,14.454200,0.000000,1.000000,0.0,0.000000,0.000000,1.000000,0.000000,1100.500000
75%,3.000000,35.750000,1.000000,0.000000,31.471875,1.000000,1.000000,0.0,0.000000,0.000000,1.000000,1.000000,1204.750000
max,3.000000,76.000000,8.000000,9.000000,512.329200,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1309.000000
